In [ ]:
import sys
from os.path import join as pjoin

In [ ]:
from collections import defaultdict

In [ ]:
from itertools import chain

In [ ]:
paths = ['/home/contematto/github/PyMVPA/',
        '/data/famface/openfmri/scripts']

In [ ]:
for p in paths:
    if p not in sys.path:
        sys.path.insert(1, p)

In [ ]:
from expdir import expdir
fns = expdir()

In [ ]:
from mvpa2.suite import *

In [ ]:
# load union mask
union_mask = fmri_dataset(fns.maskfn)
# load v1v_mask
v1v_mask = fmri_dataset('/data/famface/openfmri/ProbAtlas_v4/v1v_mh.nii.gz')

In [ ]:
def ev_mask_fn(which):
    return '/data/famface/openfmri/ProbAtlas_v4/{0}_mh.nii.gz'.format(which)

In [ ]:
def load_mask(maskfn):
    mask = fmri_dataset(maskfn)
    mask_ = (union_mask.samples + mask.samples) == 2.
    return mask_.flatten()

In [ ]:
# Setup permutation routines
def get_unique_combs(nsub):
    # assumes there are two superordinate categories, 
    # and that nsub is the total number of subordinate categories,
    # and that nsub is even
    combs = list(combinations(range(nsub), nsub/2))
    unique_combs = set()
    for c1, c2 in zip(combs, combs[::-1]):
        unique_combs.add(tuple(sorted((c1, c2))))
    return sorted(unique_combs)

def flatten(listOfLists):
    "Flatten one level of nesting"
    return list(chain.from_iterable(listOfLists))

In [ ]:
def permute_conditions(ds, permute=0):
    """Permute the conditions in ds maintaining the hierarchical structure
    of the problem. Permute is the permutation number with max permute = 34.
    Permute = 0 corresponds to the identity permutation
    
    If it finds condition_orig in ds.sa, uses that as original condition 
    to permute"""
    perm = get_unique_combs(8)[permute]
    perm = flatten(perm)
    unique_conds = np.unique(ds.sa.condition)
    mapperm = dict()
    if 'condition_orig' in ds.sa:
        ds.sa['condition'] = ds.sa.condition_orig.copy()
    else:
        ds.sa['condition_orig'] = ds.sa.condition.copy()
    for i, p in enumerate(perm):
        mapperm[unique_conds[i]] = unique_conds[p]
    for i in range(ds.nsamples):
        this_cond = ds.sa.condition[i]
        ds.sa.condition[i] = mapperm[this_cond]
    print("USING PERMUTATION {0}".format(mapperm))

In [ ]:
rois = ['v1v', 'v1d', 
        'v2v', 'v2d', 
        'v3v', 'v3d',
        'v1', 
        'v2', 
        'v3',
        'v1+v2',
        'v1+v2+v3',
       ]

In [ ]:
mask_roi = {roi: load_mask(ev_mask_fn(roi)) for roi in rois}

In [ ]:
for roi in rois:
    print('ROI {0}: {1:6d} voxels'.format(roi, mask_roi[roi].sum()))

In [ ]:
def prepare_dataset(subnr, mask=None):
    ds = h5load(fns.betafn(subnr))
    if mask is not None:
        ds = ds[:, mask]
    ds = ds[ds.sa.condition != 'self', :]
    zscore(ds, chunks_attr='chunks', dtype='float32')
    
    return ds

In [ ]:
clf = LinearCSVMC()

# feature selection helpers
fselector = FractionTailSelector(.1,  # select top 10% features
                                 tail='upper',
                                 mode='select', sort=False)
sbfs = SensitivityBasedFeatureSelection(OneWayAnova(), fselector,
                                        enable_ca=['sensitivities'])
# create classifier with automatic feature selection
fsclf = FeatureSelectionClassifier(clf, sbfs)

In [ ]:
partitioner = FactorialPartitioner(NFoldPartitioner(attr='condition'),
                                   attr='targets')

In [ ]:
cv_anova = CrossValidation(fsclf, partitioner,
                     errorfx=lambda p, t: np.mean(p == t),
                     #enable_ca=['stats'],
                     #postproc=mean_sample()
                     )

In [ ]:
df_acc_roi = pd.DataFrame({'subid': [], 
                           'roi': [], 
                           'fold':[], 
                           'acc': [],
                           'permutation': []})

In [ ]:
for isub in range(1, 34):
    print("Running for subject {0}".format(isub))
    ds = prepare_dataset(isub)
    for iperm in range(35):  # iperm = 0 is the identity permutation
        permute_conditions(ds, permute=iperm)
        # set up targets
        ds.sa['familiarity'] = ['familiar' if 'friend' in a
                                else 'control'
                                for a in ds.sa.condition]
        ds.sa['targets'] = ds.sa['familiarity']
        for roi in rois:
            ds_ = ds[:, mask_roi[roi]]
            print('    Running on ROI {0}'.format(roi))
            acc_roi = cv_anova(ds_).samples.flatten()

            nfolds = len(acc_roi)
            tmp_df = pd.DataFrame(
                {'subid': ['sub{0:03d}'.format(isub)] * nfolds,
                 'roi': [roi] * nfolds,
                 'fold': range(1, nfolds+1),
                 'acc': acc_roi,
                 'permutation': [iperm] * nfolds})
            df_acc_roi = pd.concat((df_acc_roi, tmp_df))

In [ ]:
df_ev.to_csv('ev-roi_acc.csv', index=False)